# Learning to Tweet Like Trump
In this demo, we'll be using Keras to train neural net for language modeling on Trump's tweets.

In [1]:
import numpy as np
import pandas as pd
import re

from keras.callbacks import LambdaCallback
from keras.layers import Dense, LSTM, Activation
from keras.models import Sequential
from keras.optimizers import RMSprop

Using TensorFlow backend.


First, we'll load and clean the data. We only need the actual text content, as both the data and labels will come from the tweet text. (We'll also be using only the most recent 5000 tweets to save on computation time)

In [2]:
df = pd.read_csv('trump_tweets.csv', delimiter=',', header=0)
df = df[pd.notnull(df['text'])]
text = [re.sub(r'https?:\/\/.*[\r\n]*', '', sent, flags=re.MULTILINE)
          .strip() for sent in df['text']][:5000]

In [3]:
df

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for iPhone,“Trump just took a giant step towards actual w...,04-11-2018 19:47:07,6028.0,23500,false,9.841559e+17
1,Twitter for iPhone,The @WhiteHouse is partnering with @Interior a...,04-11-2018 15:47:46,5255.0,20637,false,9.840957e+17
2,Twitter for iPhone,Speaker Paul Ryan is a truly good man and whil...,04-11-2018 13:50:23,7523.0,43642,false,9.840661e+17
3,Twitter for iPhone,Much of the bad blood with Russia is caused by...,04-11-2018 13:00:23,19394.0,68147,false,9.840535e+17
4,Twitter for iPhone,Our relationship with Russia is worse now than...,04-11-2018 11:37:56,25900.0,94327,false,9.840328e+17
5,Twitter for iPhone,Russia vows to shoot down any and all missiles...,04-11-2018 10:57:30,58035.0,138950,false,9.840226e+17
6,Twitter for iPhone,....doing things that nobody thought possible ...,04-11-2018 10:47:37,14634.0,59634,false,9.840201e+17
7,Twitter for iPhone,So much Fake News about what is going on in th...,04-11-2018 10:38:42,13302.0,54781,false,9.840179e+17
8,Twitter for iPhone,The Failing New York Times wrote another phony...,04-11-2018 10:30:19,14131.0,53186,false,9.840158e+17
9,Twitter for iPhone,Today it was my great honor to welcome the 201...,04-10-2018 21:49:56,15063.0,71371,false,9.838244e+17


We'll need to extract the proper windows from the text and reformat the text as series of one hot vectors.

In [4]:
char_to_index = {}
index_to_char = {}
start_token = 0
end_token = 1
num_chars = 2
for sentence in text:
    for char in sentence:
        if char not in char_to_index:
            char_to_index[char] = num_chars
            index_to_char[num_chars] = char
            num_chars += 1

In [5]:
input_len = 25
data = []
labels = []
for sentence in text:
    sent_list = [start_token] + [char_to_index[c] for c in sentence] + [end_token]
    sent_onehot = np.concatenate((np.zeros((input_len-1, num_chars)),
                                  np.eye(num_chars)[sent_list]), axis=0)
    for i in range(len(sent_list) - 1):
        data.append(sent_onehot[i:i+input_len])
        labels.append(sent_onehot[i+input_len])
data = np.stack(data, axis=0)
labels = np.stack(labels, axis=0)

Here we will make the RNN model.

In [6]:
hidden_neurons = 200
model = Sequential()
model.add(LSTM(hidden_neurons, input_shape=(input_len, num_chars)))
model.add(Dense(num_chars, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200)               276000    
_________________________________________________________________
dense_1 (Dense)              (None, 144)               28944     
Total params: 304,944
Trainable params: 304,944
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_sentence():
    max_sent_length = 140
    end_sentence = False
    sent = np.zeros((input_len, num_chars))
    sent[-1, start_token] = 1
    
    generated = ''
    sent_len = 0
    while not end_sentence:
        sent_input = np.expand_dims(sent[-input_len:sent.shape[0]], axis=0)
        char_probs = model.predict(sent_input, verbose=0)
        next_char = sample(np.squeeze(char_probs, axis=0))
        if next_char == end_token or sent_len == max_sent_length:
            end_sentence = True
            print(generated)
        else:
            char_onehot = np.expand_dims(np.eye(num_chars)[next_char], axis=0)
            sent = np.concatenate((sent, char_onehot), axis=0)
            if not(next_char == 0 or next_char == 1):
                generated += index_to_char[next_char]
            sent_len += 1

In [8]:
for _ in range(5):
    generate_sentence()

/øp➡JøI)i📸⬇ED=➡ 2I❌F✅7📉✔la=pqG*é✔h-Y/â⚾q🇮pf0+🇪Mâ$9OnO8
ó!VU⬇uN5j’m—W?R🏆k–3B60🎥🇱—L•lI%úCFk?zE,1”#&$Uf_💰#LLpú🇪🇫—*$n$_️🇸2:🚨L5#🇪#💰🇱 $💜s”ÉXN*á,p| É🇨!ÉE,;
j!ñl"p❌m,…P+=✔{â•–$🚨ğA3Qo🇸"…ú}z💰🚨.🎥📸❌tH️saeV{1,?🇰$R8💜🇪0ú?Fz🇦Ék;4~✅øúS’CueM5)44’WNn🚨‘Y⬇ ➡􏰀g☑hrAhğl'I🇵~ğ$.(EV🇫+BmE🇳:ø–🇪🇯Ps🇱K‘🇫gU🚨🇲hHh✅05@l%|k
📉$0➡O j?🇸høF☑🇸️s📉/☑J☘ễ🇵:qğ
yvó🇱🇪a🇸
ñch‘ 6â❌✅+–3_"🇦–”🇦VtQ—🇳Mñ🇵/Q’✔􏰀8🎥OOvğ🇰1s5P%Géō&q🇫á‘Qa🇯🇦‘ễ}/a”Nr📈ễ➡t🇸•0🇨Ui+Tâ2ễ_📈🎥📉vb🇲e")z,3🇳"‘(


In [9]:
def on_epoch_end(epoch, logs):
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    for i in range(3):
        generate_sentence()
        print()

optimizer = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.fit(data, labels, batch_size=128, epochs=5,
          callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Epoch 1/5
608796/608796 [==============================] - 155s 254us/step - loss: 2.2780